In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## import 用到的套件

In [2]:
import numpy as np
#載入繪圖套件
import matplotlib.pyplot as plt
import matplotlib.font_manager as plt_font
#載入for迴圈進度條
from tqdm import tqdm
folder="/content/drive/MyDrive/解密AI黑盒子分享/"
twfont1 = plt_font.FontProperties(fname=folder+"字型/kaiu.ttf")

In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,models,datasets
from torch.utils.data import DataLoader

## 是否有GPU可以使用

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 定義訓練集和測試集資料預處理

In [5]:
train_transform=transforms.Compose([
    transforms.RandomResizedCrop(224),#隨機裁剪
    transforms.RandomHorizontalFlip(),#隨機旋轉
    transforms.ToTensor(),#轉成Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])#像素資料歸一化
])

test_transform=transforms.Compose([
    transforms.Resize(256),#改變大小
    transforms.CenterCrop(224),#中央裁剪
    transforms.ToTensor(),#轉成Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])#像素資料歸一化
])

## 載入訓練和測試資料

In [6]:
train_dataset = datasets.ImageFolder(folder+"資料集/水果30分類/train", train_transform)
test_dataset = datasets.ImageFolder(folder+"資料集/水果30分類/test", test_transform)

In [7]:
print('訓練集圖片資料', len(train_dataset))
print('分類名稱', train_dataset.classes)
print('分類數量', len(train_dataset.classes))
n_class=len(train_dataset.classes)

訓練集圖片資料 4375
分類名稱 ['哈密瓜', '奇異果', '小黃瓜', '山竹', '柚子', '柳橙', '梨', '椰子', '楊梅', '榴槤', '檸檬', '櫻桃', '火龍果', '白葡萄', '石榴', '砂糖橘', '紅番茄', '紅葡萄', '紅蘋果', '聖女番茄', '胡蘿蔔', '芒果', '苦瓜', '草莓', '荔枝', '西瓜', '青蘋果', '香蕉', '鳳梨', '龍眼']
分類數量 30


## 分類和索引雙向對應，並存檔

In [8]:
index2label = {i:x for i,x in enumerate(train_dataset.classes) }
print(index2label)
np.save(folder+"資料集/水果30分類/index2label.npy", index2label)

{0: '哈密瓜', 1: '奇異果', 2: '小黃瓜', 3: '山竹', 4: '柚子', 5: '柳橙', 6: '梨', 7: '椰子', 8: '楊梅', 9: '榴槤', 10: '檸檬', 11: '櫻桃', 12: '火龍果', 13: '白葡萄', 14: '石榴', 15: '砂糖橘', 16: '紅番茄', 17: '紅葡萄', 18: '紅蘋果', 19: '聖女番茄', 20: '胡蘿蔔', 21: '芒果', 22: '苦瓜', 23: '草莓', 24: '荔枝', 25: '西瓜', 26: '青蘋果', 27: '香蕉', 28: '鳳梨', 29: '龍眼'}


## 定義數據加載DataLoader

In [9]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=2)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=2)

## 遷移式學習方法

### 方法一：只訓練最後一層全連階層

In [10]:
#載入預訓練模型
model=models.resnet18(weights='DEFAULT')
#更改最後一層全連階層
model.fc=nn.Linear(model.fc.in_features,n_class)
#設定優化器的更新範圍
optimizer = torch.optim.Adam(model.fc.parameters())

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

### 方法二：微調訓練所有層的參數

In [ ]:
#載入預訓練模型
model = models.resnet18(weights='DEFAULT')
#更改最後一層全連階層
model.fc = nn.Linear(model.fc.in_features,n_class)
#設定優化器的更新範圍
optimizer = torch.optim.Adam(model.parameters())

### 方法三：從頭訓練所有層的參數

In [ ]:
#載入預訓練模型
model=models.resnet18()
#更改最後一層全連階層
model.fc=nn.Linear(model.fc.in_features,n_class)
#設定優化器的更新範圍
optimizer=torch.optim.Adam(model.parameters())

In [11]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 模型訓練的配置

In [12]:
model=model.to(device)
LossFun=nn.CrossEntropyLoss()

## 開始執行訓練

In [13]:
for epoch in tqdm(range(30)):
  model.train()
  for images, labels in train_loader:
    optimizer.zero_grad()
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = LossFun(outputs, labels)
    loss.backward()
    optimizer.step()

  0%|          | 0/30 [27:44<?, ?it/s]


KeyboardInterrupt: ignored

## 用測試集資料進行測試

In [ ]:
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    correct += (torch.argmax(outputs,dim=1)==labels).sum().item()
  print("驗證資料準確度：",correct*100/len(test_dataset),"%")

100%|██████████| 137/137 [00:19<00:00,  6.93it/s]

驗證資料準確度： 94.74285714285715 %


## 保存模型

In [ ]:
torch.save(model, folder+"資料集/水果30分類/fruit30.pth")